# Tree Methods

In [4]:
import findspark

In [5]:
findspark.init('/home/oussama/spark-2.4.0-bin-hadoop2.7')

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [9]:
data = spark.read.csv('College.csv', inferSchema=True, header=True)

In [10]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [11]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [14]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
from pyspark.ml.feature import StringIndexer

In [17]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndexer')

In [18]:
output_fixed = indexer.fit(output).transform(output)

In [19]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndexer: double (nullable = false)



In [21]:
final_data = output_fixed.select('features', 'PrivateIndexer')

In [22]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [23]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                       GBTClassifier,
                                      RandomForestClassifier)

In [24]:
from pyspark.ml import Pipeline

In [36]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndexer', featuresCol='features')
rfc = RandomForestClassifier(numTrees=150, labelCol='PrivateIndexer', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndexer', featuresCol='features')

In [43]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [44]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [45]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
my_Binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndexer')


In [47]:
print('DTC')
print(my_Binary_eval.evaluate(dtc_preds))

DTC
0.9320416994492525


In [48]:
print('RFC')
print(my_Binary_eval.evaluate(rfc_preds))

RFC
0.9636113296616828


In [33]:
print('GBT')
print(my_Binary_eval.evaluate(gbt_preds))

GBT
0.9134047993705745


In [34]:
my_Binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndexer',
                                               rawPredictionCol='prediction')

In [51]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [54]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndexer',
                                            metricName='accuracy')

In [55]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [56]:
rfc_acc

0.9336283185840708